In [16]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path
import regex as re

# DeepL

In [17]:
path = Path('/home/hombresabio/AI/Climate Project/DataPrep/dataset')


In [18]:
data = pd.read_csv(path / 'dataset_5.csv', index_col='id')
data_translated = pd.read_csv(path / 'dataset_5_translated.csv', index_col='id')
data= data[~data.index.isin(data_translated.index)]
len(data)

0

In [7]:
data['len'] = data['text'].apply(lambda x : len(x))
data['len'].sum()

977374

In [11]:
dfs = np.array_split(data, 2)

for i, df in enumerate(dfs):
    with open(f'f{i}.txt', 'w',encoding="utf-8") as f:
      f.write( " ******* ".join( [ "_" + str(index) + '_ ' + row['text'] for index, row in df.iterrows()]))


In [12]:
# Open a file: file
text_list = []
for i in range(2):
  with open(f'f{i}USpl.txt',mode='r',encoding="utf-8") as file:
    text = file.read()
    text = re.sub("_\d+_", "", text)
    text_list.extend(re.split("\*\*\*\*\*\*\*", text))
    file.close()

In [14]:
len(text_list), len(data.index)

(151, 151)

In [15]:
data["translated"] = text_list
data.drop(columns="len", inplace=True)
data.to_csv("translated_1.csv")

# Google translate

In [23]:
from translate import Translator
from nltk import sent_tokenize

def back_translation(text,source = 'polish', target = 'english'):
    translator = Translator(to_lang= target)
    tokenized_src = sent_tokenize(text, source)
    text_tr = [translator.translate(sentence) for sentence in tokenized_src]
    text_tr = ' '.join([i for i in text_tr if i is not None])
    tokenized_tr = sent_tokenize(text_tr, target)
    translator = Translator(to_lang=source)
    translated = [translator.translate(sentence) for sentence in tokenized_tr]
    translated = ' '.join([i for i in translated if i is not None])
    return translated


In [26]:
dfs = np.array_split(data, 20)

In [27]:
from multiprocessing import Pool

for index, df in tqdm(enumerate(dfs)):
    with Pool(5) as pool:
        df['translated'] = pool.map(back_translation, df['text'])
    df.to_csv(f'translated{12+index}.csv')

0it [00:00, ?it/s]

In [36]:
dfs = np.array_split(data, 12)


for index, df in tqdm(enumerate(dfs)):
    if index in range(0,11):
        print(index)
    df['translated'] = df['text'].apply(back_translation)
    df.to_csv(f'translated{index}.csv')


0it [00:00, ?it/s]

0


0it [00:05, ?it/s]


RateLimitError: You exceeded your current quota, please check your plan and billing details.

In [13]:
data = pd.concat([pd.read_csv(f'translated{i}.csv', index_col='id') for i in range(11)])
data[data.index.duplicated()]

3